### Notas

1. los numeros de resolucion y las fechas son importantes porque hay resoluciones mas nuevas que dejan sin efecto resoluciones nteriores. Ej: 11-11-24.pdf  ARTÍCULO 1°- Dejar sin efecto en todos sus términos las Resolución N° RESOL-2024-3144-INSSJP-DE#INSSJ

In [1]:
import pandas as pd

In [2]:
with open('data/txt/01-02-24.txt', 'r') as file:
    content = file.read()
#print(content)

In [3]:
content = content.lower()

In [5]:
#print(content)

1. Parsear boletin oficial en Resoluciones, disposiciones, etc.
2. Keywords extract, filtrar el contenido no relevante.
3. Summarize content
4. Agregar categoria

In [28]:
import os

def load_files(dir):
   """
    Loads all .txt files from a folder and stores them in a dictionary.
    :param dir: Path to the folder containing the .txt files.
    :return: Dictionary with file names as keys and content as values.
    """
    files = {}
    for file in os.listdir(dir):
        if file.endswith(".txt"):
            path = os.path.join(dir, file)
            with open(path, 'r', encoding='utf-8') as f:
                files[file] = f.read()
    return files

In [29]:
dir_txt = "data/txt"
files = load_files(dir_txt)

In [30]:
#files

In [87]:
def divide_sections(text, delimiter="•"):
     """
    Divides a text into sections using a delimiter.
    :param text: The full text to be divided.
    :param delimiter: The pattern used to separate sections (default is empty lines).
    :return: A list of sections.
    """
    return text.split(delimiter)

In [88]:
sections_per_file = {name: divide_sections(content) for name, content in files.items()}

In [89]:
# example
for name, sections in sections_per_file.items():
    print(f"File: {name}")
    print(f"Sections found: {len(sections)}")
    print(f"First section: {sections[0][:9000]}...\n")
    break

File: 30-07-24.txt
Sections found: 14
First section: BOLETÍN DEL INSTITUTO

Índice

RESOLUCIONES

Dirección Ejecutiva

...



In [90]:
# Mostrar más ejemplos y contenido extenso
num_files = 1
num_sections = 1 # per file
max_chars = 1

for idx, (name, sections) in enumerate(sections_per_file.items()):
    print(f"🔹 **File {idx + 1}: {name}**")
    print(f"Sections found: {len(sections)}\n")
    
    # algunas secciones del archivo
    for i, section in enumerate(sections[:num_sections]):
        print(f"  ✂️ Section {i + 1}:\n{section[:max_chars]}...")
        if len(section) > max_chars:
            print(f"[...] (Contenido truncado: {len(section)} caracteres)\n")
        else:
            print("\n")
    
    print("=" * 80 + "\n")
    
    if idx + 1 >= num_files:  # Limitar el número de archivos mostrados
        break


🔹 **File 1: 30-07-24.txt**
Sections found: 14

  ✂️ Section 1:
B...
[...] (Contenido truncado: 66 caracteres)




### Filtrar secciones
Una vez que tengo las secciones parseadas (delimitadas), quiero encontrar las secciones que tienen determinados keywords para despues filtrarlas.

In [91]:
def filter_sections_keyword(sections, keywords):
    """
    Filter sections that contain certain keywords
    :param sections: List of sections.
    :param keywords: List of keywords to search for in the text.
    :return: List of sections that contain any of the keywords.
    """
    keywords = [word.lower() for word in keywords]  # Convertir palabras clave a minúsculas
    filtered_sections = [
        section for section in sections 
        if any(word in section.lower() for word in keywords)
    ]
    return filtered_sections

In [92]:
keywords = ["designar", "designa"]

In [93]:
# create dict comprehension

filtered_section_by_file = {name: filter_sections_keyword(sections, keywords) for name, sections in sections_per_file.items()}

In [94]:
def show_filtered_sections(filtered_section_by_file, max_files=1):
    for idx, (name, filtered_sections) in enumerate(filtered_section_by_file.items()):
        if idx >= max_files:
            break
        print(f"🔹 File: {name}")
        print(f"Sections that contain keywords: {len(filtered_sections)}\n")
        for i, section in enumerate(filtered_sections[:2]): # up to 3 examples per file
            print(f"✂️ Section {i+1}:\n{section[:300]}...")
            print("[...]\n")
        print("="*80 + "\n")

In [102]:
#show_filtered_sections(filtered_section_by_file, max_files=3)

#### Save filtered sections to txt

In [103]:
def save_sections_to_individual_files(section_dict, output_folder):
    """
    Save sections from a dictionary into individual .txt files.
    :param section_dict: Dictionary where keys are filenames and values are lists of sections.
    :param output_folder: Folder to save individual .txt files.
    """
    os.makedirs(output_folder, exist_ok=True)
    for file_name, sections in section_dict.items():
        file_path = os.path.join(output_folder, f"{file_name}.txt")
        with open(file_path, "w", encoding="utf-8") as f:
            for i, section in enumerate(sections):
                f.write(f"Section {i + 1}:\n{section.strip()}\n{'-' * 80}\n")

# Save filtered sections to individual files
save_sections_to_individual_files(filtered_section_by_file, "data/filtered_sections")

In [96]:
#with open('filtered_sections.txt', 'w') as file:
#    for key, value in filtered_section_by_file.items():
#        file.write(f"{key}: {value}\n")

#### Save rest of sections to txt

In [97]:
def exclude_sections_keyword(sections, keywords):
    """
    Filter sections that DO NOT contain certain keywords.
    :param sections: List of sections.
    :param keywords: List of keywords to search for in the text.
    :return: List of sections that do not contain any of the keywords.
    """
    keywords = [word.lower() for word in keywords]  # Convert keywords to lowercase
    excluded_sections = [
        section for section in sections 
        if not any(word in section.lower() for word in keywords)
    ]
    return excluded_sections

In [100]:
# create dict comprehension
excluded_section_by_file = {name: exclude_sections_keyword(sections, keywords) for name, sections in sections_per_file.items()}

In [104]:
# Save filtered sections to individual files
save_sections_to_individual_files(excluded_section_by_file, "data/unfiltered_sections")

In [101]:
#with open('unfiltered_sections.txt', 'w') as file:
#    for key, value in excluded_section_by_file.items():
#        file.write(f"{key}: {value}\n")